In [ ]:
#default_exp utils.math.compute_pillars_legacy

# Compute pillars legacy

> First step is computing the all necessary information regarding the pillars on the given point cloud. As source we used the original implementation in second (https://github.com/traveller59/second.pytorch/tree/master/second) as well as another implementation in tensorflow (https://github.com/fferroni/PointPillars). The main idea is to devide the point cloud into descrete sections, namly pillars, and calculate different attributes for each point in regards to the pillar it is in. These attributes include the distance to the pillar mean location as well as the distance to the pillar center.

In [ ]:
import torch
import sys
import numba
from numba import jit
import numpy as np
from configparser import SafeConfigParser

#sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/")
sys.path.insert(0, "../pointpillars")

### CUDA

Set the best cuda device to improve calculation speed

In [ ]:
dt = torch.float
dev = "cuda:0" if torch.cuda.is_available() else "cpu"
torch.cuda.set_device(dev)
#dev = "cpu"

## Data structure

Firstly, we want to familiarize ourselves with the structure of the binary files we receive from the pseudo-lidar convertion.

In [ ]:
file_location = "/home/qhs67/git/bachelorthesis_sven_thaele/code/data/kitti/training/pseudo_lidar/002000.bin"

In [ ]:
pc_np = np.fromfile(file_location, dtype=np.float32, count=-1)
pc = torch.as_tensor(pc_np, device=dev)
pc = pc.reshape([-1,4])
pc

In [ ]:
x = pc[:,0]
y = pc[:,1]
z = pc[:,2]
r = pc[:,3]
x, y, z, r

## Helping methods
> In this section we want to create different helper methods structures our code.
### Timing

In [ ]:
#hide
def time_method(method, runs=1, kwargs=None):

    times = torch.empty(runs, dtype=torch.float)
    for i in range(runs-1):
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)
        start.record()

        if kwargs:
            method(**kwargs)
        else:
            method()

        end.record()
        torch.cuda.synchronize()
        times[i] = start.elapsed_time(end)

    return times.mean()

In [ ]:
#export
import torch
from pointpillars.utils.io import read_config

##  Preprocessing
> Methods to get the point cloud in the correct shape
### Removing points outside of area

Remove invalid points according to the settings given in the config file




## Calculation
> Next we implement the pillar calculation itself.
### Calculate the pillar center

In [ ]:
#export
def calculate_pillar_center(x_min_pillar, y_min_pillar, gparam):

    x = x_min_pillar + 0.5 * gparam.getfloat('x_step')
    y = x_min_pillar + 0.5 * gparam.getfloat('y_step')
    z = (gparam.getfloat('z_max') - gparam.getfloat('z_min')) / 2.0

    return torch.tensor([x, y, z], dtype=dt, device=dev)

In [ ]:
x_min_pillar = 10
y_min_pillar = 20
gparam = read_config()['pillars']

calculate_pillar_center(x_min_pillar, y_min_pillar, gparam)

### Pillar class

In [ ]:
#hide
class Pillar():
    points = None
    center = None
    mean_values = None
    points_with_attributes = None
    max_points_per_pillar = read_config()['pillars'].getint('max_points_per_pillar')

    def __init__(self, points, center):
        if not torch.is_tensor(points) or not torch.is_tensor(center):
            raise ValueError("Tensor expected but not given.")

        if points.shape[1] != 4 or center.shape[0] != 3:
            raise ValueError("Unexpected array shape.")

        points.to(dev)
        center.to(dev)

        points_trimmed = self.sample_tensor(points, self.max_points_per_pillar)
        self.points = points_trimmed
        self.center = center

    def _calculate_point_attributes(self):
        # drop reflectance
        points_diff_to_mean = (self.points - self.mean_values)[:, 0:3].to(dev)

        # calculate point difference to center
        # drop z coordinate and reflectance
        points_diff_to_center = (self.points - torch.cat((self.center, torch.tensor([0], device=dev))))[:, 0:2].to(dev)
        pcloud_with_attributes = torch.cat((self.points, points_diff_to_mean, points_diff_to_center), dim=1)

        return pcloud_with_attributes.to(dev)

    def calculate_obj_attributes(self):
        """
            A method to save computation time
        """
        x_mean = torch.mean(self.points[:,0])
        y_mean = torch.mean(self.points[:,1])
        z_mean = torch.mean(self.points[:,2])

        self.mean_values = torch.tensor([x_mean, y_mean, z_mean, 0], device=dev)
        self.points_with_attributes = self._calculate_point_attributes()
        self.points_with_attributes = self.pad_tensor(self.points_with_attributes, self.max_points_per_pillar)

    @staticmethod
    def pad_tensor(tensor, length, mask=(False, False, False, True)):
        if len(tensor) < length:
            # ensures that the padding mask is long enough, we only want to pad the rows
            mask_pad = []
            for bool_pad in mask:
                mask_pad.append(length - tensor.shape[0] if bool_pad else 0)
            tensor = torch.nn.functional.pad(tensor, tuple(mask_pad))

        try:
            return tensor.to(dev)
        except AttributeError:
            return tensor

    @staticmethod
    def sample_tensor(tensor, length):
        while len(tensor) > length:
            rand_index = torch.randint(len(tensor), (1,))
            if torch.is_tensor(tensor):
                tensor = torch.cat([tensor[0:rand_index], tensor[rand_index+1:]]).to(dev)
            else:
                tensor.pop(rand_index)

        try:
            return tensor.to(dev)
        except AttributeError:
            return tensor


## Calculate the pillars itself

Finally, we can compute the pillars itself.
As return we want to receive two numpy tensors.
The first contains all the points from the cloud corresponding to each pillar.
Also included are the newly calculated attributes for each point itself.
In the second tensor we store the centers for each pillar.
The reason for that is, so we can move the resulting pillar feature maps back to the correct location in 3d-space.

In [ ]:
#export
def calculate_pillars(pcloud):
    if not torch.is_tensor(pcloud):
        raise ValueError("Tensor expected but not given.")

    gparam = read_config()['pillars']
    pcloud = remove_invalid(pcloud, gparam)

    x_min = gparam.getfloat('x_min')
    y_min = gparam.getfloat('y_min')
    x_step = gparam.getfloat('x_step')
    y_step = gparam.getfloat('y_step')

    max_points_per_pillar = gparam.getint('max_points_per_pillar')
    max_pillars = gparam.getint('max_pillars')

    # TODO: Specify what happens when n_x and n_y are not exact ints
    n_x = int((gparam.getfloat('x_max') - x_min) / x_step)
    n_y = int((gparam.getfloat('y_max') - y_min) / y_step)


    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()



    pillars = []
    for i in range(n_x):
        for j in range(n_y):
            x_min_pillar = x_min + i * x_step
            x_max_pillar = x_min_pillar + x_step
            y_min_pillar = y_min + j * y_step
            y_max_pillar = y_min_pillar + x_step

            points_in_pillar = get_points_in_pillar(pcloud, x_min_pillar, x_max_pillar, y_min_pillar, y_max_pillar)
            if points_in_pillar.shape[0] == 0:
                continue

            pillar_center = calculate_pillar_center(x_min_pillar, y_min_pillar, gparam)
            # TODO: Move Pillar object to gpu
            pillars.append(Pillar(points_in_pillar, pillar_center))

    end.record()
    torch.cuda.synchronize()
    print(start.elapsed_time(end))

    # cannot add zeroes to pillars array
    pillars_points = torch.empty((max_pillars, max_points_per_pillar, 9), dtype=torch.float, device=dev)
    pillars_indicies = torch.empty((max_pillars, 3), dtype=torch.float, device=dev)
    pillars_trimmed = Pillar.sample_tensor(pillars, max_pillars)
    pillerid = 0
    for pillar in pillars_trimmed:
        pillar.calculate_obj_attributes()
        center = pillar.center
        points_with_attributes = pillar.points_with_attributes

        pillars_points[pillerid] = points_with_attributes
        pillars_indicies[pillerid] = center
        pillerid += 1

    return pillars_points, pillars_indicies

In [ ]:
arr1 = torch.rand(1200,270000,10)
sys.getsizeof(arr1)

In [ ]:
import time

pillars_points, pillars_indicies = calculate_pillars(pc)
pillars_cfg = read_config()['pillars']
#pillars_points, pillars_points.shape, pillars_indicies, pillars_indicies.shape, start.elapsed_time(end)
out_points, out_indicies = calculate_pillars(pc)
out_points, out_points.shape, time_method(calculate_pillars, runs=1, kwargs={'pcloud': pc})